**This notebook is an exercise in the [Feature Engineering](https://www.kaggle.com/learn/feature-engineering) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/creating-features).**

---


# Introduction #

In this exercise you'll start developing the features you identified in Exercise 2 as having the most potential. As you work through this exercise, you might take a moment to look at the data documentation again and consider whether the features we're creating make sense from a real-world perspective, and whether there are any useful combinations that stand out to you.

Run this cell to set everything up!

In [42]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.feature_engineering_new.ex3 import *

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
# 모델 검증을 위한 교차 검증 함수를 가져옵니다.
from xgboost import XGBRegressor
# XGBoost 회귀 모델


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


# Prepare data
df = pd.read_csv("../input/fe-course-data/ames.csv")
X = df.copy()
y = X.pop("SalePrice")

-------------------------------------------------------------------------------

Let's start with a few mathematical combinations. We'll focus on features describing areas -- having the same units (square-feet) makes it easy to combine them in sensible ways. Since we're using XGBoost (a tree-based model), we'll focus on ratios and sums.

# 1) Create Mathematical Transforms

Create the following features:

- `LivLotRatio`: the ratio of `GrLivArea` to `LotArea`
- `Spaciousness`: the sum of `FirstFlrSF` and `SecondFlrSF` divided by `TotRmsAbvGrd`
- `TotalOutsideSF`: the sum of `WoodDeckSF`, `OpenPorchSF`, `EnclosedPorch`, `Threeseasonporch`, and `ScreenPorch`

In [43]:
# YOUR CODE HERE
X_1 = pd.DataFrame()  # dataframe to hold new features

X_1["LivLotRatio"] = df.GrLivArea / df.LotArea
X_1["Spaciousness"] = (df.FirstFlrSF + df.SecondFlrSF) /df.TotRmsAbvGrd
X_1["TotalOutsideSF"] = df.WoodDeckSF + df.OpenPorchSF + df.EnclosedPorch + df.Threeseasonporch + df.ScreenPorch


# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [44]:
# Lines below will give you a hint or solution code
q_1.hint()
#q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Your code should look something like:
```python
X_1["LivLotRatio"] = ____ / ____
X_1["Spaciousness"] = (____ + ____) / ____
X_1["TotalOutsideSF"] = ____ + ____ + ____ + ____ + ____
```


<span style="color:#3366cc">Hint:</span> Your code should look something like:
```python
X_1["LivLotRatio"] = ____ / ____
X_1["Spaciousness"] = (____ + ____) / ____
X_1["TotalOutsideSF"] = ____ + ____ + ____ + ____ + ____
```


-------------------------------------------------------------------------------

If you've discovered an interaction effect between a numeric feature and a categorical feature, you might want to model it explicitly using a one-hot encoding, like so:

```
# One-hot encode Categorical feature, adding a column prefix "Cat"
X_new = pd.get_dummies(df.Categorical, prefix="Cat")

# Multiply row-by-row
X_new = X_new.mul(df.Continuous, axis=0)

# Join the new features to the feature set
X = X.join(X_new)
```

# 2) Interaction with a Categorical

We discovered an interaction between `BldgType` and `GrLivArea` in Exercise 2. Now create their interaction features.

In [45]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', 'FirstFlrSF', 'SecondFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF',

In [46]:
df.BldgType.value_counts()

BldgType
OneFam      2425
TwnhsE       233
Duplex       109
Twnhs        101
TwoFmCon      62
Name: count, dtype: int64

In [47]:
# 숫자 특성과 범주형 특성 간의 상호 작용 효과를 발견했다면, 
# 이를 명시적으로 모델링하고 싶을 수 있습니다. 
# 이를 위해 원-핫 인코딩을 사용

# YOUR CODE HERE
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
X_2 = pd.get_dummies(df.BldgType, prefix="Bldg") # >> 이진 열로 변환
print(X_2)
print()

# Multiply
X_2 = X_2.mul(df.GrLivArea, axis = 0)
# Multiply: 원-핫 인코딩된 BldgType 열과 GrLivArea 열을 요소별로 곱함
print("\nMultiplication result:")
print(X_2)

# Check your answer
q_2.check()

      Bldg_Duplex  Bldg_OneFam  Bldg_Twnhs  Bldg_TwnhsE  Bldg_TwoFmCon
0           False         True       False        False          False
1           False         True       False        False          False
2           False         True       False        False          False
3           False         True       False        False          False
4           False         True       False        False          False
...           ...          ...         ...          ...            ...
2925        False         True       False        False          False
2926        False         True       False        False          False
2927        False         True       False        False          False
2928        False         True       False        False          False
2929        False         True       False        False          False

[2930 rows x 5 columns]


Multiplication result:
      Bldg_Duplex  Bldg_OneFam  Bldg_Twnhs  Bldg_TwnhsE  Bldg_TwoFmCon
0             0.0       165

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [48]:
# Lines below will give you a hint or solution code
#q_2.hint()
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

X_2 = pd.get_dummies(df.BldgType, prefix="Bldg")
X_2 = X_2.mul(df.GrLivArea, axis=0)

```

# 3) Count Feature

Let's try creating a feature that describes how many kinds of outdoor areas a dwelling has. Create a feature `PorchTypes` that counts how many of the following are greater than 0.0:

```
WoodDeckSF
OpenPorchSF
EnclosedPorch
Threeseasonporch
ScreenPorch
```

In [49]:
df.WoodDeckSF

0       210.0
1       140.0
2       393.0
3         0.0
4       212.0
        ...  
2925    120.0
2926    164.0
2927     80.0
2928    240.0
2929    190.0
Name: WoodDeckSF, Length: 2930, dtype: float64

In [50]:
# 주거지에 있는 야외 공간의 종류를 설명하는 특성
# "WoodDeckSF" 열에 하나의 행에 대한 값이 120.0이라는 것은
# 해당 행에 있는 주거지의 나무 데크 면적이 120.0 square feet임을 의미
X_3 = pd.DataFrame()

# YOUR CODE HERE
X_3["PorchTypes"] = df[[
   "WoodDeckSF",
"OpenPorchSF",
"EnclosedPorch",
"Threeseasonporch",
"ScreenPorch"
]].gt(0.0).sum(axis=1)
# .gt(0.0): 선택된 열들에 대해 0.0보다 큰 값을 가진 행


# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [51]:
# Lines below will give you a hint or solution code
q_3.hint()
q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Your code should look someting like:
```python
X_3 = pd.DataFrame()

X_3["PorchTypes"] = df[[
    ____,
    ____,
    ____,
    ____,
    ____,
]].____.sum(axis=1)
```


<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

X_3 = pd.DataFrame()

X_3["PorchTypes"] = df[[
    "WoodDeckSF",
    "OpenPorchSF",
    "EnclosedPorch",
    "Threeseasonporch",
    "ScreenPorch",
]].gt(0.0).sum(axis=1)

```

# 4) Break Down a Categorical Feature

`MSSubClass` describes the type of a dwelling:

In [52]:
df.MSSubClass.unique()

array(['One_Story_1946_and_Newer_All_Styles', 'Two_Story_1946_and_Newer',
       'One_Story_PUD_1946_and_Newer',
       'One_and_Half_Story_Finished_All_Ages', 'Split_Foyer',
       'Two_Story_PUD_1946_and_Newer', 'Split_or_Multilevel',
       'One_Story_1945_and_Older', 'Duplex_All_Styles_and_Ages',
       'Two_Family_conversion_All_Styles_and_Ages',
       'One_and_Half_Story_Unfinished_All_Ages',
       'Two_Story_1945_and_Older', 'Two_and_Half_Story_All_Ages',
       'One_Story_with_Finished_Attic_All_Ages',
       'PUD_Multilevel_Split_Level_Foyer',
       'One_and_Half_Story_PUD_All_Ages'], dtype=object)

You can see that there is a more general categorization described (roughly) by the first word of each category. Create a feature containing only these first words by splitting `MSSubClass` at the first underscore `_`. (Hint: In the `split` method use an argument `n=1`.)

In [56]:
'''
"MSSubClass" 열에서 각 카테고리의 첫 번째 단어만 포함하는 
새로운 특성을 만들어야 합니다. 이를 위해서는 "MSSubClass" 열의 각 값을 
첫 번째 밑줄 (_) 기준으로 분리해야 합니다. 
분리할 때는 split 메서드의 인자로 n=1을 사용
'''
X_4 = pd.DataFrame()

# YOUR CODE HERE
X_4["MSClass"] = df.MSSubClass.str.split("_", n=1, expand = True)[0]
# 첫 번째 밑줄 (_)을 기준으로 분리
# n=1은 첫 번째 밑줄만을 기준으로 하겠다는 의미
# expand=True: 이 옵션을 설정하면 분리된 결과를 여러 개의 열로 반환
# [0] :  첫 번째 열만을 선택하여 가져오는 작업

# Check your answer
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [54]:
# Lines below will give you a hint or solution code
#q_4.hint()
q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

X_4 = pd.DataFrame()

X_4["MSClass"] = df.MSSubClass.str.split("_", n=1, expand=True)[0]

```

# 5) Use a Grouped Transform

The value of a home often depends on how it compares to typical homes in its neighborhood. Create a feature `MedNhbdArea` that describes the *median* of `GrLivArea` grouped on `Neighborhood`.

In [64]:
X_5 = pd.DataFrame()

# YOUR CODE HERE
# X_5["MedNhbdArea"] = df.groupby('Neighborhood')['GrLivArea'].median()
# Neighborhood, GrLivArea 이 두개의 열의 인덱스가 일치하지 않기 때문
# groupby 후에 transform 또는 agg 함수를 사용
X_5["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")

# Check your answer
q_5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [61]:
# Lines below will give you a hint or solution code
#q_5.hint()
q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

X_5 = pd.DataFrame()

X_5["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")

```

Now you've made your first new feature set! If you like, you can run the cell below to score the model with all of your new features added:

In [ ]:
X_new = X.join([X_1, X_2, X_3, X_4, X_5])
score_dataset(X_new, y)

# Keep Going #

[**Untangle spatial relationships**](https://www.kaggle.com/ryanholbrook/clustering-with-k-means) by adding cluster labels to your dataset.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/feature-engineering/discussion) to chat with other learners.*